In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

<img src="https://monkeylearn.com/static/6700dcab9bcc691104dd0d794f6e7ef4/Sentiment-analysis-of-Twitter-Social.png" length="100px" width="1000px">

# Twitter sentiment analysis - Extracting emotions through machine learning
* Sentiment analysis is the automatic manner of analyzing textual content and sorting it into sentiments positive, negative, or neutral. Sentiment analysis is used to investigate opinions in Twitter data and it can assist organizations to analyse and understand about their brand. Twitter Sentiment Analysis is a stream of Social network analysis. This is also known as opinion mining

* We import numpy, pandas, matplotlib, and seaborn for data analysis and visualization
* We import wordcloud for EDA
* We import re, string, nltk, spacy for tweets cleaning
* We import sklearn and Xgboost for machine learning and metrics

In [ ]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# setting up the background style for the plots
plt.style.use('fivethirtyeight')

import re
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import nltk
from nltk.stem.porter import PorterStemmer

import spacy
nlp = spacy.load('en')

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix

In [ ]:
train_df = pd.read_csv('../input/twitter-sentiment-analysis-hatred-speech/train.csv')
print('Train dataset shape: ', train_df.shape)
print('Train dataset columns: ', train_df.columns)

In [ ]:
#selecting necessary features from train data
train_df = train_df[['label','tweet']]

### Exploratory Data Analysis

In [ ]:
#Labbel Categories count
sns.countplot(x= train_df['label'])
plt.title('Label Counts')
plt.show()

* by the above plot we can understand that in our data most of the data labelled as 0 with means positive sentiment, on the other hand label = 1 means negative sentiment.

In [ ]:
from wordcloud import WordCloud

sentences = train_df['tweet'].tolist()
sentences_ss = " ".join(sentences)

plt.figure(figsize=(10,10))
plt.imshow(WordCloud().generate(sentences_ss))

* With the help of wordcloud we can find most repeated words easily from the whole dataset. By the above image we can come to a conclusion that in our data 'user' is the word which repeated more number of times. And we have some unknown symbols also in our data. So, befor proceeding with modelling, we should first clean the data.

### Word Frequency Analysis

In [ ]:
train_df['tweet'].apply(len).plot(bins=100, kind = 'hist')
plt.title('Length frequency of all tweets')
plt.show()

* The above histogram shows us the length frequency of the each tweet from the dataset

### Spliting dataset into input(X) and target(y)

In [ ]:
X = train_df.iloc[:,[1]]
y = train_df.iloc[:,0]

### Cleaning of Tweets

In this cleaning process we are going to do the following steps
* removing @usernames 
* converting all words into lowercase
* removing numbers
* removing punctuations
* removing URLs
* tokanize each tweet -> Tokenization is a way of separating a piece of text (sentence) into smaller units called tokens.
* remove while spaces
* remove stop words -> “stop words” usually refers to the most common words in a language, like is, are, the, a, an, and so on
* removing non alphabetics tokens
* performing lemmatization -> remove inflectional endings only i.e. running, runner == run

In [ ]:
# Remove @usernames
X['tweet'] = X['tweet'].map(lambda x: re.sub('@\S+', ' ', x))

#all lower case
X['tweet']=X['tweet'].map(lambda x: x.lower())

# Remove numbers
X['tweet'] = X['tweet'].map(lambda x: re.sub(r'\d+', ' ', x))

# Remove Punctuation
X['tweet']  = X['tweet'].map(lambda x: x.translate(x.maketrans('', '', string.punctuation)))

# Remove urls
url_cleaner = "https?:\S+|http?:\S|[^A-Za-z0-9]+"
X['tweet'] = X['tweet'].map(lambda x: re.sub(url_cleaner, ' ', x))

# Remove white spaces
X['tweet'] = X['tweet'].map(lambda x: x.strip())

# Tokenizer
X['tweet'] = X['tweet'].map(lambda x: word_tokenize(x))

# Remove stop words
stop_words = set(stopwords.words('english'))
X['tweet'] = X['tweet'].map(lambda x: [w for w in x if not w in stop_words])

# Remove non alphabetic tokens
X['tweet'] = X['tweet'].map(lambda x: [w for w in x if w.isalpha()])

# turning back to string
X['tweet'] = X['tweet'].map(lambda x: ' '.join(x))

#Lemmatizer
X['tweet']=X["tweet"].apply(lambda x: " ".join([w.lemma_ for w in nlp(x)]))

### Processing the Data

In [ ]:
import gensim

documents = [_text.split() for _text in X.tweet] 
w2v_model = gensim.models.word2vec.Word2Vec(size=200, 
                                            window=5, 
                                            min_count=3, 
                                            workers=6)
w2v_model.build_vocab(documents)

In [ ]:
words = w2v_model.wv.vocab.keys()
vocab_size = len(words)
print("Vocab size", vocab_size)

### Finding Similar words

In [ ]:
w2v_model.train(documents, total_examples=len(documents), epochs=30)

In [ ]:
#Test word embeddings
w2v_model.most_similar("job")

In [ ]:
train_corpus = []

for i in range(0, 31962):
  review = re.sub('[^a-zA-Z]', ' ', X['tweet'][i])
  review = review.split()
  
  ps = PorterStemmer()
  
  # stemming
  review = [ps.stem(word) for word in review]
  
  # joining them back with space
  review = ' '.join(review)
  train_corpus.append(review)

### CountVectorizer
* CountVectorization is usedd to create a sparse matrix representing all the words in the dataset

In [ ]:
cv = CountVectorizer(max_features = 3000)
x = cv.fit_transform(train_corpus).toarray()

print(x.shape)
print(y.shape)

### Sentiment Analysis

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size = 0.20, random_state = 46)

sc = StandardScaler()

x_train = sc.fit_transform(x_train)
x_val = sc.transform(x_val)

In [ ]:
model = DecisionTreeClassifier()
model.fit(x_train, y_train)

y_pred_dt = model.predict(x_val)

print("Training Accuracy :", model.score(x_train, y_train))
print("Validation Accuracy :", model.score(x_val, y_val))


In [ ]:
model = RandomForestClassifier()
model.fit(x_train, y_train)

y_pred_rf = model.predict(x_val)

print("Training Accuracy :", model.score(x_train, y_train))
print("Validation Accuracy :", model.score(x_val, y_val))

In [ ]:
model = XGBClassifier()
model.fit(x_train, y_train)

y_pred_xg = model.predict(x_val)

print("Training Accuracy :", model.score(x_train, y_train))
print("Validation Accuracy :", model.score(x_val, y_val))

In [ ]:
# calculating the f1 score for the validation set
print("f1 score :", f1_score(y_val, y_pred_rf))

# confusion matrix
cm = confusion_matrix(y_val, y_pred_rf)
print(cm)

### Conclusion:
* Twitter Sentiment analysis helps to monitor and analyse the emotions of human beings based on their tweets. 
* In this project work, Random forest classifier, Decission tree classifier, and XGBoost classifiers are used to perform Twitter sentiment analysis, out of these 3 algorithms Random Forest classifier works well and it prediction score is 95.13%.
* nltk and Spacy has been used in this project to do the natural language processing job.
* In future, we can extract data directly from twitter through api and we can analyze with the random forest.
* We can also improve accuracy by doing prameter tuning using gridsearcv or randomizedsearchcv

### Reference:

* https://github.com/sharmaroshan/Twitter-Sentiment-Analysis/blob/master/Twitter_Sentiment.ipynb
* https://www.kaggle.com/pavansanagapati/knowledge-graph-nlp-tutorial-bert-spacy-nltk
* https://towardsdatascience.com/twitter-sentiment-analysis-classification-using-nltk-python-fa912578614c
* https://monkeylearn.com/blog/sentiment-analysis-of-twitter/